In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GroupShuffleSplit
from imblearn.over_sampling import SMOTE
from pathlib import Path

In [2]:
# for testing modularized functions
df_path = "../../grp_own/mean_with_labels.csv" # edit this to select dataset
df = pd.read_csv(df_path)

In [3]:
def remove_columns(df: pd.DataFrame):
    '''
    Removes transcript_name and gene_id columns.
    Add or remove columns as required.
    '''
    df1 = df.drop(["transcript_name", "gene_id", "nucleotide_seq"], axis=1)
    return df1

def create_vectorizer(df:pd.DataFrame, n):
    '''
    to vectorize nucleotide sequences using corpus from training dataset
    df should be training dataset, n would be value for ngrams
    '''
    corpus = df['nucleotide_seq']
    vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(n,n)).fit(corpus)
    return vectorizer

def trigram_tokenize(df:pd.DataFrame, vectorizer):
    '''
    to apply trigram on column "nucleotide_seq" using TfidfVectorizer
    '''
    v_nucleotide_seq = vectorizer.transform(df["nucleotide_seq"])
    v = pd.DataFrame(v_nucleotide_seq.toarray()) # convert sparse matrix to array

    # creating dictionary to easily add vectorized sequence features as columns to dataframe
    new_nucleotide_data = dict() 
    for i in range(v.shape[1]):
        key = "s" + str(i)
        new_nucleotide_data[key] = v.iloc[:,i]

    df_final = df.assign(**new_nucleotide_data)
    df_final = df_final.fillna(0)
    
    return df_final #returns dataframe with vectorized nucleotide features as columns

def data_split(df:pd.DataFrame):
    '''
    splits data by gene into train and test sets according to the percentage given.
    '''
    X = df.drop(["label"], axis=1)
    y = df["label"]

    gss = GroupShuffleSplit(n_splits=2, random_state=0, test_size=0.2)
    train_i, test_i = next(gss.split(X,y,groups=X.gene_id))

    X_train = X.loc[train_i]
    y_train = y.loc[train_i]

    X_test = X.loc[test_i]
    y_test = y.loc[test_i]

    return X_train, X_test, y_train, y_test

def create_standardizer(df:pd.DataFrame):
    '''
    create standardizer based on training dataset to standardize other datasets
    df should be X_train
    '''
    # Identify numerical and categorical columns
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = df.select_dtypes(exclude=['int64', 'float64']).columns

    standardizer = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', 'passthrough', categorical_cols)  # Leave categorical columns unchanged
        ])
    
    standardizer.fit(df)

    return standardizer

def standardize_data(df:pd.DataFrame, standardizer):
    '''
    Standardizes numerical features while leaving non-numerical features unchanged using fitted preprocesser from create_standardizer function
    '''
    x_columns = df.columns
    df_scaled = standardizer.transform(df)
    df_scaled = pd.DataFrame(df_scaled, columns = x_columns)
    
    return df_scaled

def synthetic_oversampling(X_train, y_train):
    '''
    Uses SMOTE to oversample the minority class.
    '''
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    return X_train_resampled, y_train_resampled


# Stringing together 

In [4]:
def pipeline_all(df):
    """
    Purpose: Setting up the pipeline? to drop unnecessary columns, followed by vectorizing, splitting, standardising and oversampling
    """
    # Train-test split and remove the "transcript_name", "gene_id", "nucleotide_seq" columns
    Xtrain, Xtest, ytrain, ytest = data_split(df)

    # Vectorization
    vectorizer = create_vectorizer(Xtrain,3)
    Xtrain_v = trigram_tokenize(Xtrain, vectorizer)

    # Remove the "transcript_name", "gene_id", "nucleotide_seq" columns
    Xtrain_v = remove_columns(Xtrain_v)

    # Scaling Xtrain_v and Xtest_v with Scaler fitted on Xtrain
    standardizer = create_standardizer(Xtrain_v)
    Xtrain_scaled = standardize_data(Xtrain_v, standardizer)

    # SMOTE on training set 
    Xtrain_resampled, ytrain_resampled = synthetic_oversampling(Xtrain_scaled, ytrain)
    
    return vectorizer, standardizer, Xtrain_resampled, Xtest, ytrain_resampled, ytest

In [5]:
vectorizer, standardizer, X_train, X_test, y_train, y_test = pipeline_all(df)
X_train.to_csv("../../dataset/Xtrain.csv", index = False)
y_train.to_csv("../../dataset/ytrain.csv", index = False)
X_test.to_csv("../../dataset/Xtest.csv", index = False)
y_test.to_csv("../../dataset/ytest.csv", index = False)

Creating Model and Model Training

In [19]:
# Importing training sets 
X_train = pd.read_csv("../../dataset/Xtrain.csv")
y_train = pd.read_csv("../../dataset/ytrain.csv")
y_train = y_train['label']

# Intiailise model

# uncomment/ comment to decide which model to use
# model = RandomForestClassifier()
model = XGBClassifier() 

model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

Predicting Test Datasets 0, 1 and 2

In [22]:
# Read in test dataset to make predictions on

data_file_path = "../../dataset/d1_mean_final.csv"
# data_file_path = "../../dataset/d2_mean_final.csv"
# data_file_path = "../../dataset/Xtest.csv"
X_test = pd.read_csv(data_file_path)


# Process test data: vectorizing, removing unwanted columns, scaling

# Vectorization with vectorizer fitted on training data
X_test_v = trigram_tokenize(X_test, vectorizer)

# Remove the "transcript_name", "gene_id" (for dataset0), "nucleotide_seq" columns
# X_test_v = remove_columns(X_test_v) #for Xtest from dataset0, cause theres gene id for that
X_test_v = X_test_v.drop(["transcript_name", "nucleotide_seq"], axis=1)

# Scaling Xtest_v with Scaler fitted on training data
X_test_scaled = standardize_data(X_test_v, standardizer)

# Prediction
# y_pred = model.predict(X_test_scaled) # class predictions
y_pred_prob = model.predict_proba(X_test_scaled) # predicted probabilities for classes 0 and 1
pred_for_classes = pd.DataFrame(y_pred_prob, columns= ["no_modification_probability", "modified_probability"])
y_pred_m6a = pred_for_classes["modified_probability"]

# Merged prediction back to original dataframe (pre processing) to format final output of transcriptname, transcript position, score
merged = pd.concat([X_test, y_pred_m6a], axis=1)
final = merged[["transcript_name", "json_position", "modified_probability"]]
final.rename(columns={'transcript_name':'transcript_id', 'json_position': 'transcript_position', 'modified_probability': 'score'}, inplace=True)


# stem attribute extracts the file
# name
name = Path(data_file_path).stem
model_name = model.__class__.__name__
results_file_path = "../../prediction/"+model_name+"_prediction_"+name+".csv"
final.to_csv(results_file_path, index=False)

C:\Users\yilin\AppData\Local\Temp\ipykernel_49796\3398385458.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.rename(columns={'transcript_name':'transcript_id', 'json_position': 'transcript_position', 'modified_probability': 'score'}, inplace=True)
